# Simple ML Jump (STUDENT VERSION)
Billy Hau - June 29, 2022

- Data Cleaning / Feature Engineering / Modeling 
- Deployment

In [1801]:
# Import Libraries
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import multiprocessing as mp

In [1802]:
# Read CSV File
df = pd.read_csv("./data.csv")
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,21.340134,15.113852,-1.000000,-1.000000,False,True,True,False
1,42.000000,15.413610,57.000000,14.252161,False,False,False,False
2,38.155075,15.413610,53.444794,14.252161,False,True,False,False
3,34.056351,15.413610,49.654922,14.252161,False,True,False,False
4,30.188143,15.413610,46.078190,14.252161,False,True,False,False
...,...,...,...,...,...,...,...,...
19747,35.707039,14.488118,50.152767,15.764210,False,True,False,False
19748,31.860840,14.488118,45.967796,15.764210,False,True,False,False
19749,28.224371,14.488118,42.011032,15.764210,False,True,False,False
19750,24.600929,14.488118,38.068443,15.764210,False,True,False,False


In [1803]:
# What do each of the columns mean?

In [1804]:
jump = df[df['Jump'] == True]
dead = jump['Dead'].to_numpy()
# dead.value_counts()
for i in range(dead.shape[0]):
    # print(jump.index[i])
    try:
        if any(df['Dead'][jump.index[i] + x] for x in range(0, 11)):
            df.drop(df[df.index == jump.index[i]].index, inplace=True)
    except:
        pass
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,21.340134,15.113852,-1.000000,-1.000000,False,True,True,False
1,42.000000,15.413610,57.000000,14.252161,False,False,False,False
2,38.155075,15.413610,53.444794,14.252161,False,True,False,False
3,34.056351,15.413610,49.654922,14.252161,False,True,False,False
4,30.188143,15.413610,46.078190,14.252161,False,True,False,False
...,...,...,...,...,...,...,...,...
19747,35.707039,14.488118,50.152767,15.764210,False,True,False,False
19748,31.860840,14.488118,45.967796,15.764210,False,True,False,False
19749,28.224371,14.488118,42.011032,15.764210,False,True,False,False
19750,24.600929,14.488118,38.068443,15.764210,False,True,False,False


In [1805]:
dead = df['Dead'].to_numpy()
# dead.value_counts()
for i in range(df.shape[0]):
    # print(jump.index[i])
    try:
        if any(df['Dead'][df.index[i] + x] for x in range(0, 6)):
            print(df.index[i] + 5)
            df.drop(df[df.index == df.index[i]].index, inplace=True)
    except:
        pass
df


5
48
50
52
112
114
116
118
120
241
243
245
247
249
268
270
272
274
276
288
290
292
294
394
396
398
400
668
670
672
674
676
678
737
739
771
773
775
777
813
815
817
819
853
855
857
942
944
946
965
967
986
988
1024
1026
1262
1264
1266
1268
1325
1327
1329
1456
1458
1460
1561
1563
1565
1567
1673
1675
1677
1679
1681
1687
1689
1691
1693
1695
1697
1703
1705
1707
1709
1711
1713
1715
1717
1864
1866
1868
1870
1872
1874
1903
1905
1907
1909
1984
1986
1989
1991
1993
1995
1997
1999
2085
2087
2135
2137
2139
2141
2162
2164
2166
2203
2205
2207
2209
2211
2213
2215
2236
2238
2240
2242
2244
2246
2303
2305
2307
2326
2328
2330
2392
2394
2396
2398
2409
2411
2413
2415
2450
2452
2454
2456
2458
2490
2492
2494
2496
2529
2531
2533
2570
2572
2574
2592
2594
2698
2700
2702
2704
2706
2814
2816
2818
2820
2822
2855
2857
2859
2862
2864
2866
2868
2870
2888
2890
2892
2894
2911
2913
2915
2923
2925
2927
2930
2932
2934
2936
2938
2940
2942
2944
2946
2948
3061
3063
3065
3067
3069
3107
3109
3111
4379
4381
4383
4385
4387
4389
439

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
1,42.000000,15.413610,57.000000,14.252161,False,False,False,False
2,38.155075,15.413610,53.444794,14.252161,False,True,False,False
3,34.056351,15.413610,49.654922,14.252161,False,True,False,False
4,30.188143,15.413610,46.078190,14.252161,False,True,False,False
5,26.079243,15.413610,42.278908,14.252161,False,True,False,False
...,...,...,...,...,...,...,...,...
19747,35.707039,14.488118,50.152767,15.764210,False,True,False,False
19748,31.860840,14.488118,45.967796,15.764210,False,True,False,False
19749,28.224371,14.488118,42.011032,15.764210,False,True,False,False
19750,24.600929,14.488118,38.068443,15.764210,False,True,False,False


In [1806]:
# Are there data we don't care about?
df.drop(df[df['Dead'] == True].index, inplace=True)
df.drop(df[df['Grounded'] == False].index, inplace=True)
df = df.drop(["Grounded", "Dead", "Scored"], inplace=False, axis=1)
df

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump
2,38.155075,15.413610,53.444794,14.252161,False
3,34.056351,15.413610,49.654922,14.252161,False
4,30.188143,15.413610,46.078190,14.252161,False
5,26.079243,15.413610,42.278908,14.252161,False
6,22.223995,15.413610,38.714161,14.252161,False
...,...,...,...,...,...
19747,35.707039,14.488118,50.152767,15.764210,False
19748,31.860840,14.488118,45.967796,15.764210,False
19749,28.224371,14.488118,42.011032,15.764210,False
19750,24.600929,14.488118,38.068443,15.764210,False


In [1807]:
jumpCount = df[df['Jump'] == True].shape[0]
jumpCount

381

In [1808]:
noJumpCount = df[df['Jump'] == False].shape[0]
noJumpCount

11012

In [1809]:
df_toDrop = df[df['Jump'] == False].sample(noJumpCount - jumpCount)
df.drop(df_toDrop.index, inplace=True)

In [1810]:
# Create the Input DataFrame
X = df[['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance', 'Bar 2 Speed']]
X = X.to_numpy()

In [1811]:
# Create the Output DataFrame and Map True / False to 1 / 0
Y = df[['Jump']]


def bool2Num(row):
    if row['Jump']:
        return 1
    else:
        return 0


Y['Jump'] = Y.apply(bool2Num, axis='columns')
Y = Y['Jump'].to_numpy()

/tmp/ipykernel_1114684/2096279593.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Jump'] = Y.apply(bool2Num, axis='columns')


In [1812]:
# Train Test Split with test size set to 20%
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [1813]:
from sklearn.linear_model import LogisticRegression

# Define and Build Model (Logistic Regression)
model = RandomForestClassifier()
model.fit(x_train, y_train)


RandomForestClassifier()

In [1814]:
predict = model.predict(x_test)

In [1815]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [1816]:
# Print Out Prediction Accuraccy
accuracy_score(y_test, predict)

0.9285714285714286

In [1817]:
# Try a Different Model (Random Forest)
"""model2 = ________________________________
model2.__________________________

predict = ___________________________
accuracy_score(_________________, ________________)"""


'model2 = ________________________________\nmodel2.__________________________\n\npredict = ___________________________\naccuracy_score(_________________, ________________)'

# Deployment

In [1818]:
# Command to Open SIMPLE ML Jump 2 (for Example)

# Windows OS
#env_path = "D:\\User\\Desktop\\10botics Data Science\\ML Game\\SimpleMLJump2 Builds\\Windows\\Simple ML Jump 2.exe"

# MacOS
#env_path = 'open -n "/Users/billwaa/Desktop/ML Game/Simple ML Jump 2.app"' 

env_path = "/home/wcyat/dev/kaggle/code/ml-jump/Linux/SimpleMLJump2.x86_64"

In [1819]:
# Import Libraries
import socket
import struct
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import time
import subprocess
import platform
import collect
import multiprocessing as mp


# Find Open Socket Ports
def findOpenSockets(size: int):
    sock = []
    port = []

    for i in range(size):
        sock.append(socket.socket())
        sock[i].bind(('localhost', 0))
        port.append(sock[i].getsockname()[1])

    for i in range(len(sock)):
        sock[i].close()

    return port


# Find Open Ports and Launch Game Environment
envNum = 1
openPorts = findOpenSockets(envNum * 2)
gamePort = openPorts[0]
apiPort = openPorts[1]

# Determine OS and Launch 
if platform.system() == 'Darwin':
    env_path += f" --args --apiPort {str(apiPort)} --gamePort {str(gamePort)} --small false"
    process = subprocess.Popen([env_path], shell=True)
else:
    process = subprocess.Popen([env_path, '--apiPort', str(apiPort), '--gamePort', str(gamePort), '--small', 'false'])

# Establish UDP Network Client
localIP = "localhost"
bufferSize = 1024
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPServerSocket.bind((localIP, apiPort))

# Set UDP Timeout
UDPServerSocket.settimeout(3)

# Wait for Game Environment to Open
t0 = time.time()

while (time.time() - t0 < 5):
    pass


# Extract Data
def extractData(data):
    dist1 = struct.unpack('f', data[:4])
    speed1 = struct.unpack('f', data[4:8])
    dist2 = struct.unpack('f', data[8:12])
    speed2 = struct.unpack('f', data[12:16])

    onGround = True if (data[16] & (1 << 0)) == 1 else False
    toJump = True if (data[17] & (1 << 0)) == 1 else False
    isDead = True if (data[18] & (1 << 0)) == 1 else False
    scored = True if (data[19] & (1 << 0)) == 1 else False

    dat = [dist1[0], speed1[0], dist2[0], speed2[0], toJump, onGround, isDead,
           scored]  # Swap Order to Accomadate Old Model
    df_temp = pd.DataFrame(dat).transpose()

    return df_temp


# Reset Simulation
UDPServerSocket.sendto(bytes.fromhex('07 00'), (localIP, gamePort))

2

In [1820]:
df2 = pd.DataFrame()

# Process Loop - Retrieve Data from Simulation, Run Through Model, Output Command
while True:
    try:
        bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
        t0 = time.time()
        message = bytesAddressPair[0]
        address = bytesAddressPair[1]
        df_temp = extractData(message)
        X = df_temp.iloc[:, :4]
        X = X.astype(float)
    # Data PreProcessing Function Go Here
    # Model Prediction Function Go Here
        predict = model.predict(X)
        print(predict, end='\r')
    # If Jump Decision is Made, Send to Game
        if predict[0] > 0.5:
            # Jump Command
            UDPServerSocket.sendto(bytes.fromhex(
                '06 01'), (localIP, gamePort))
        if df_temp.iloc[:, 6][0]:
            UDPServerSocket.sendto(bytes.fromhex(
                '07 00'), (localIP, gamePort))
            df2.columns = ['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance',
                           'Bar 2 Speed', 'Jump', 'Grounded', 'Dead', 'Scored']
            try:
                pd.concat([pd.read_csv("data.csv"),
                           df2]).to_csv("data.csv", index=False)
            except:
                df2.to_csv("data.csv", index=False)

            df2 = pd.DataFrame()

        df2 = pd.concat([df2, df_temp])
    except Exception as e:
        print(e)
        break


timed out


In [1821]:
# Export Data to CSV
df2.columns = ['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance', 'Bar 2 Speed', 'Jump', 'Grounded', 'Dead', 'Scored']
try:
    pd.read_csv("data.csv").append(df2).to_csv("data.csv", index=False)
except:
    df2.to_csv("data.csv", index=False)

/tmp/ipykernel_1114684/97692202.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd.read_csv("data.csv").append(df2).to_csv("data.csv", index=False)


In [1822]:
# Close UDP Port When Game is Closed
UDPServerSocket.close()